In [ ]:
!pip install --upgrade --force-reinstall pymupdf
!pip install langdetect
!pip install PyPDF2

Traduz e salva artigo traduzido sem as referências (pula artigo caso arquivo da tradução exista)

In [25]:
import time
import fitz  # PyMuPDF
import json
import requests
import sys
import os

GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

API_KEY = input("Digite sua chave API da Groq (ou deixe em branco para pular traduções): ")
if not API_KEY:
  print("⚠️ ATENÇÃO: API_KEY não definida.")
  sys.exit()

with open('artigos_info.json', 'r', encoding='utf-8') as f:
  artigos_info = json.load(f)

try:
  for idx, artigo in enumerate(artigos_info):

    caminho_pdf = artigo['pdf']
      
    caminho_artigo_traduzido = 'artigos_traduzidos_pt/'+ os.path.basename(caminho_pdf).replace('.pdf', '_traduzido.txt')
    print(caminho_artigo_traduzido)

    if not os.path.isfile(caminho_artigo_traduzido):
        try:
            with fitz.open(caminho_pdf) as doc:
                texto = ""
                for pagina in doc:
                    texto += pagina.get_text()
            texto = texto.strip()
        except Exception as e:
            print(f"Erro ao extrair PDF {caminho_pdf}: {e}")
            sys.exit()
    
        # Remove referencias
        referencias = texto.split("References")
        if len(referencias) > 1:
          texto = "References".join(texto.split("References")[:-1]);
    
        referencias = texto.split("Bibliography")
        if len(referencias) > 1:
          texto = "Bibliography".join(texto.split("Bibliography")[:-1]);
    
        tamanho_maximo = 2000
        
        # Divide o texto em partes menores
        palavras = texto.split()
        partes = []
        parte_atual = ""
          
        for palavra in palavras:
            if len(parte_atual) + len(palavra) + 1 <= tamanho_maximo:
                parte_atual += (" " if parte_atual else "") + palavra
            else:
                if '.' in parte_atual:
                  # Divide a parte atual nas frases com ponto
                  lastCaracter = parte_atual[-1]
                  frases = parte_atual.split('.')
    
                  parte_valida = '.'.join(frases[:-1]) + '.' if len(frases) > 1 else frases[0] + '.'
                  restante = frases[-1]  # última frase incompleta
                  # if(restante):
                  #   parte_valida += '.'
                  if len(restante) + len(palavra) + 1 >= tamanho_maximo:
                    partes.append(parte_valida)
                    partes.append(restante +  ('.' if lastCaracter == '.' else ""))
                    parte_atual = palavra
                  else:
                    partes.append(parte_valida)
                    parte_atual = restante + " " + palavra
                else:
                    partes.append(parte_atual)
                    parte_atual = palavra
    
        if parte_atual:
            partes.append(parte_atual)
            
        print(f"🔄{idx} Dividido em {len(partes)} partes para tradução")
    
    
    
        # Traduz cada parte e junta
        traducoes = []
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        }
    
        for idx, parte in enumerate(partes):
            print(f"🌍 Traduzindo parte {idx+1}/{len(partes)}...")
            prompt = (
                "Traduza o seguinte texto científico para português do Brasil. "
                "A tradução deve ser natural, precisa e manter o sentido original. "
                "A resposta deve conter apenas o texto traduzido, sem comentários, explicações ou marcações extras."
                "Mantenha termos técnicos quando apropriado.:\n\n"
                f"{parte}"
            )
    
            payload = {
                "model": "llama3-8b-8192",  # ou outro modelo disponível na Groq
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.2
            }
    
            # Taxa de limitação para não sobrecarregar a API
            time.sleep(10)
    
            try:
                response = requests.post(GROQ_API_URL, headers=headers, json=payload)
                response.raise_for_status()
                traducao = response.json()["choices"][0]["message"]["content"]
                print(traducao)
                traducoes.append(traducao)
                print("✅ Parte traduzida.")
            except Exception as e:
                print(f"❌ Erro na tradução da parte {idx+1}: {e}")
                time.sleep(30)  # Espera mais tempo em caso de erro
                sys.exit()
    
        texto_traduzido = "\n".join(traducoes)
        nome_arquivo = os.path.basename(caminho_pdf).replace('.pdf', '_traduzido.txt')
        with open("artigos_traduzidos_pt/" + nome_arquivo, 'w', encoding='utf-8') as f:
            f.write(texto_traduzido)
        print(f"📝 Tradução salva em: {nome_arquivo}")
        print(f"✅ Tradução concluída: {len(texto_traduzido)} caracteres")
    else:
        print("PDF: " + caminho_pdf + " já traduzido, pulando..")


except Exception as e:
  print(f"Erro ao processar artigo: {e}")

Digite sua chave API da Groq (ou deixe em branco para pular traduções):  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


artigos_traduzidos/A Large Dataset of Spontaneous Speech with the Accent Spoken in São Paulo for Automatic Speech Recognition Evaluation_traduzido.txt
PDF: ./articles/A Large Dataset of Spontaneous Speech with the Accent Spoken in São Paulo for Automatic Speech Recognition Evaluation.pdf já traduzido, pulando..
artigos_traduzidos/A Transformer-Based Tabular Approach to Detect Toxic Comments_traduzido.txt
PDF: ./articles/A Transformer-Based Tabular Approach to Detect Toxic Comments.pdf já traduzido, pulando..
artigos_traduzidos/An Ensemble of LLMs Finetuned with LoRA for NER in Portuguese Legal Documents_traduzido.txt
PDF: ./articles/An Ensemble of LLMs Finetuned with LoRA for NER in Portuguese Legal Documents.pdf já traduzido, pulando..
artigos_traduzidos/Aroeira A Curated Corpus for the Portuguese Language with a Large Number of Tokens_traduzido.txt
PDF: ./articles/Aroeira A Curated Corpus for the Portuguese Language with a Large Number of Tokens.pdf já traduzido, pulando..
artigos_tr